# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
df_encoded = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])
df_encoded.dropna(inplace=True)

## Com o dataframe ja limpo removendo colunas com dados faltantes (esses dados faltantes implicam em falhas nos modelos e metodos utilizados)

### Aqui eu separo a base de dados em treino e teste

In [5]:
df_treino, df_teste = train_test_split(df_encoded, test_size=0.25, random_state=4)

### No codigo abaixo eu gero o modelo e como pede o enunciado, gero um modelo para cada valor de alpha solicitado e com o L1_wt = 0 que representa o metodo *RIDGE*

In [6]:
modelo = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia"
md = smf.ols(modelo, data = df_treino)
reg_alpha000 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0)

reg_alpha001 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.001)

reg_alpha005 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.005)

reg_alpha010 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.01)

reg_alpha050 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.05)

reg_alpha100 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.1)

### Aqui eu avalio o R² de cada modelo usando um loop *for*

In [8]:
reg = {
    'modelo_alpha_0.000': reg_alpha000,
    'modelo_alpha_0.001': reg_alpha001,
    'modelo_alpha_0.005': reg_alpha005,
    'modelo_alpha_0.01': reg_alpha010,
    'modelo_alpha_0.05': reg_alpha050,
    'modelo_alpha_0.1': reg_alpha100
}
for y in reg:
    y_pred_teste = reg[y].predict(df_teste)
    y_pred_treino = reg[y].predict(df_treino)
    y_pred_treino_exp = np.exp(y_pred_treino)
    y_pred_teste_exp = np.exp(y_pred_teste)
    print(f'R² do modelo nos dados de TREINO: ({y}) {r2_score(df_treino['renda'], y_pred_treino_exp):.4f}')
    print(f"R² do modelo nos dados de TESTE: ({y}) {r2_score(df_teste['renda'], y_pred_teste_exp):.4f}")
    print("-" * 60 )
    

R² do modelo nos dados de TREINO: (modelo_alpha_0.000) 0.2662
R² do modelo nos dados de TESTE: (modelo_alpha_0.000) 0.2795
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.001) 0.2674
R² do modelo nos dados de TESTE: (modelo_alpha_0.001) 0.2839
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.005) 0.2733
R² do modelo nos dados de TESTE: (modelo_alpha_0.005) 0.3050
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.01) 0.2731
R² do modelo nos dados de TESTE: (modelo_alpha_0.01) 0.3240
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.05) -0.1335
R² do modelo nos dados de TESTE: (modelo_alpha_0.05) -0.2968
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.1) -6.3154
R² do modelo nos dado

### Avaliando os modelos acima e se baseando no R² dos testes, o modelo com aplha = 0.01 teve o melhor resultado

## 3. Agora fazendo para um modelo LASSO

In [14]:
modelo = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia"
md = smf.ols(modelo, data = df_treino)
reg_alpha000 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0)

reg_alpha001 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.001)

reg_alpha005 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.005)

reg_alpha010 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.01)

reg_alpha050 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.05)

reg_alpha100 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.1)

In [19]:
reg = {
    'modelo_alpha_0.000': reg_alpha000,
    'modelo_alpha_0.001': reg_alpha001,
    'modelo_alpha_0.005': reg_alpha005,
    'modelo_alpha_0.01': reg_alpha010,
    'modelo_alpha_0.05': reg_alpha050,
    'modelo_alpha_0.1': reg_alpha100
}

for y in reg:
    y_pred_teste = reg[y].predict(df_teste)
    y_pred_treino = reg[y].predict(df_treino)
    y_pred_treino_exp = np.exp(y_pred_treino)
    y_pred_teste_exp = np.exp(y_pred_teste)
    print(f'R² do modelo nos dados de TREINO: ({y}) {reg[y].rsquared_adj:.4f}')
    print(f"R² do modelo nos dados de TESTE: ({y}) {r2_score(df_teste['renda'], y_pred_teste_exp):.4f}")
    print("-" * 60 )

R² do modelo nos dados de TREINO: (modelo_alpha_0.000) 0.3590
R² do modelo nos dados de TESTE: (modelo_alpha_0.000) 0.2795
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.001) 0.3594
R² do modelo nos dados de TESTE: (modelo_alpha_0.001) 0.2796
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.005) 0.3551
R² do modelo nos dados de TESTE: (modelo_alpha_0.005) 0.2812
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.01) 0.3486
R² do modelo nos dados de TESTE: (modelo_alpha_0.01) 0.2837
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.05) 0.3458
R² do modelo nos dados de TESTE: (modelo_alpha_0.05) 0.2880
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.1) 0.3458
R² do modelo nos dados d

### Avaliando os modelos acima, o que obteve um melhor R² na base de testes foi o modelo com alpha de 0.05 analizando apenas o R²!!

## 4. Agora fazendo um *Stepwise*

### Aqui abaixo eu carrego o codigo de stepwise_selection fornecido pelo material do curso, cujo foi retirado e adaptado do fórum [*stackovervlow*](https://datascience.stackexchange.com/questions/937/does-scikit-learn-have-a-forward-selection-stepwise-regression-algorithm), da resposta do David Dale.

In [53]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # ba ckward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# variaveis = stepwise_selection(X, y)

# print('resulting features:')
# print(variaveis)

### Faço a limpeza dos dados, detalhe que todas as variaveis devem ser do tipo inteiro!

In [ ]:
X, y = df_encoded.drop(columns='renda'), df_encoded['renda']
X = pd.get_dummies(X, drop_first=True, dtype=int)
X['posse_de_imovel'] = X['posse_de_imovel'].astype(int)
X['posse_de_veiculo'] = X['posse_de_veiculo'].astype(int)

In [78]:
X_encoded = pd.get_dummies(df_encoded, drop_first=True, dtype=int)
X_encoded['posse_de_imovel'] = X_encoded['posse_de_imovel'].astype(int)
X_encoded['posse_de_veiculo'] = X_encoded['posse_de_veiculo'].astype(int)

Ao executar o comando obtemos os resultados alocados em uma variavel chamada 'variaveis'

In [ ]:
variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)X_encoded

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 0.0
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 1.75299e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  idade                          with p-value 1.9605e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior completo     with p-value 3.07164e-06
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo']
Add  qt_pessoas_residencia          with p-value 0.00747762
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'qt_pessoas_residencia']
Add  posse_de_imovel                with p-value 0.0138522
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'qt_pessoas_residencia', 'posse_d

In [77]:
X_encoded

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,tipo_renda_Pensionista,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,False,True,0,26,6.602740,1.0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
1,True,True,0,28,7.183562,2.0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,True,True,0,35,0.838356,2.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,False,True,1,30,4.846575,3.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,True,False,0,33,4.293151,1.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,True,False,0,32,9.849315,2.0,1,0,1,0,...,1,0,0,0,0,1,0,0,0,0
14995,False,True,0,48,13.887671,1.0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
14997,True,True,0,45,7.832877,2.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14998,True,False,0,36,4.298630,2.0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [79]:
reg = smf.ols("np.log(renda) ~ tempo_emprego + sexo_M + tipo_renda_Empresário + idade + Q('educacao_Superior completo') + qt_pessoas_residencia + posse_de_imovel", X_encoded)